În acest notebook se aplică modelele Random Forest și XGBoost pe datele de intrare ce conțin atributele extrase de la toate vocalele împreună. (Se consideră doar atributele rezultate dupa selecție și/sau MFDFA)



# Importurile necesare

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score


# Data

Aici se vor încărca datele dintr-un fișier csv și se vor considera doar atributele selectate.

In [2]:
names_of_features_selected = [
    'a_A',	'b_A',	'c_A',	'd_A',	'g_A',
    'a_E',	'b_E',	'c_E',	'd_E',	'g_E',
    'a_I',	'b_I',	'c_I',	'd_I',	'g_I',
    'a_O',	'b_O',	'c_O',	'd_O',	'g_O',
    'a_U',	'b_U',	'c_U',	'd_U',	'g_U',
 'mfcc_mean_1_A', 'mfcc_mean_2_A', 'mfcc_mean_4_A', 'mfcc_mean_9_A',
       'mfcc_mean_10_A', 'mfcc_mean_12_A', 'mfcc_mean_13_A',
       'loudness_slidingwindow_mean_A', 'hnr_A', 'LPC_2_A', 'LPC_3_A',
       'mfcc_mean_1_E', 'mfcc_mean_3_E', 'mfcc_mean_5_E', 'mfcc_mean_6_E',
       'mfcc_mean_7_E', 'mfcc_mean_10_E', 'mfcc_mean_11_E', 'mfcc_mean_12_E',
       'mfcc_mean_13_E', 'f0_contour_mean_E', 'f0_mean_E', 'hnr_E', 'LPC_2_E',
       'LPC_3_E', 'mfcc_mean_1_I', 'mfcc_mean_3_I', 'mfcc_mean_5_I',
       'mfcc_mean_6_I', 'mfcc_mean_7_I', 'mfcc_mean_9_I', 'mfcc_mean_11_I',
       'mfcc_mean_12_I', 'mfcc_mean_13_I', 'f0_contour_mean_I', 'hnr_I',
       'LPC_2_I', 'LPC_3_I', 'LPC_4_I', 'mfcc_mean_3_O', 'mfcc_mean_4_O',
       'mfcc_mean_6_O', 'mfcc_mean_8_O', 'mfcc_mean_9_O', 'mfcc_mean_13_O',
       'f0_contour_mean_O', 'f0_std_O', 'log_energy_O',
       'loudness_slidingwindow_mean_O', 'mfcc_mean_2_U', 'mfcc_mean_3_U',
       'mfcc_mean_5_U', 'mfcc_mean_6_U', 'mfcc_mean_7_U', 'mfcc_mean_8_U',
       'mfcc_mean_10_U', 'mfcc_mean_11_U', 'hnr_U'
]

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Read in data and display first 5 rows
features = pd.read_csv('/content/drive/MyDrive/Parkinson CSV files/features_mfdfa_and_surfboard.csv', index_col = 0)
features.head(5)

,filename,a_A,b_A,c_A,d_A,g_A,a_E,b_E,c_E,d_E,...,f2_U,f3_U,f4_U,hnr_U,LPC_0_U,LPC_1_U,LPC_2_U,LPC_3_U,LPC_4_U,label
0,AVPEPUDEAC0001a1.wav,0.005134,2.773804,14.464413,0.239951,0.438907,0.008326,2.786644,37.006651,0.358826,...,855.923600,2691.467349,3799.771303,25.690630,1.0,-1.621395,0.552271,-0.207879,0.282346,0
1,AVPEPUDEAC0001a2.wav,0.004287,2.786766,12.529315,0.228076,0.363295,0.008697,2.842495,33.157190,0.324737,...,773.168390,2650.082362,3589.815036,23.842138,1.0,-1.545521,0.431359,-0.192440,0.311552,0
2,AVPEPUDEAC0001a3.wav,0.003158,2.556353,14.310374,0.237569,0.358898,0.009720,3.256971,27.910551,0.321402,...,2802.277666,3767.566577,5145.967325,31.139092,1.0,-2.009604,1.339565,-0.627501,0.301435,0
3,AVPEPUDEAC0003a1.wav,-0.023849,1.425489,40.734144,0.501434,1.056339,0.002281,2.183830,37.554326,0.604607,...,2192.993042,3346.745084,5069.993537,16.921561,1.0,-1.657522,0.385750,0.235394,0.039457,0
4,AVPEPUDEAC0003a3.wav,-0.005259,1.808647,44.468063,0.677819,1.073030,0.002380,2.212522,40.705040,0.588844,...,2181.058987,3110.540368,5115.181137,9.182765,1.0,-1.827665,0.620723,0.272772,-0.063501,0


In [5]:
features.shape

(300, 237)

In [6]:
features.describe()

,a_A,b_A,c_A,d_A,g_A,a_E,b_E,c_E,d_E,g_E,...,f2_U,f3_U,f4_U,hnr_U,LPC_0_U,LPC_1_U,LPC_2_U,LPC_3_U,LPC_4_U,label
count,300.000000,300.000000,3.000000e+02,300.000000,300.000000,300.000000,300.000000,3.000000e+02,300.000000,300.000000,...,300.000000,300.000000,300.000000,300.000000,300.0,300.000000,300.000000,300.000000,300.000000,300.000000
mean,-0.008734,1.853691,2.284436e+08,0.238086,1625.087691,0.005324,2.246910,8.694759e+04,0.299265,661.894578,...,2213.383335,3364.068262,4350.549023,24.267637,1.0,-1.856773,1.055407,-0.478051,0.283577,0.500000
std,0.036419,1.473201,3.592656e+09,0.296372,2245.329537,0.009121,0.686928,1.298174e+06,0.348969,1686.983173,...,798.614968,538.785086,537.391857,5.016546,0.0,0.436620,0.906219,0.633758,0.226669,0.500835
min,-0.322316,0.370999,6.294143e+00,0.014113,0.025000,-0.016647,0.820592,1.681373e+01,0.019524,0.114321,...,340.917697,835.252970,2588.085080,8.552106,1.0,-2.930491,-1.330728,-2.345076,-0.365529,0.000000
25%,-0.006294,1.057812,2.164679e+01,0.081891,0.585237,0.000457,1.691353,3.296069e+01,0.097478,0.723567,...,2047.361451,2982.396001,3907.612482,21.546299,1.0,-2.128041,0.374789,-0.903242,0.153280,0.000000
50%,-0.000740,1.415724,1.533366e+02,0.179801,3.373377,0.002529,2.221574,4.861453e+01,0.223551,1.075740,...,2510.263888,3349.601574,4363.799198,24.626921,1.0,-1.889966,1.038269,-0.414444,0.303052,0.500000
75%,0.003055,2.088369,7.577553e+04,0.304539,3442.761824,0.007125,2.714854,1.342097e+02,0.360959,3.062028,...,2736.175942,3743.208708,4741.863145,27.503237,1.0,-1.560929,1.651819,0.029660,0.433558,1.000000
max,0.079384,15.458712,6.193825e+10,2.607913,7681.101526,0.040510,5.054322,2.246164e+07,3.005611,8565.258561,...,4172.477511,4582.823503,5810.900016,35.495991,1.0,-0.397971,3.424443,1.160434,0.861644,1.000000


In [7]:
features['label']

0      0
1      0
2      0
3      0
4      0
      ..
295    1
296    1
297    1
298    1
299    1
Name: label, Length: 300, dtype: int64

In [8]:
features.groupby('label').count()

,filename,a_A,b_A,c_A,d_A,g_A,a_E,b_E,c_E,d_E,...,f1_U,f2_U,f3_U,f4_U,hnr_U,LPC_0_U,LPC_1_U,LPC_2_U,LPC_3_U,LPC_4_U
label,,,,,,,,,,,,,,,,,,,,,
0,150,150,150,150,150,150,150,150,150,150,...,150,150,150,150,150,150,150,150,150,150
1,150,150,150,150,150,150,150,150,150,150,...,150,150,150,150,150,150,150,150,150,150


In [9]:
labels = np.array(features['label'])
# Remove the labels from the features
# axis 1 refers to the columns
features = features[np.intersect1d(features.columns, names_of_features_selected)]



In [10]:
names_mfdfa = [
    'a_A',	'b_A',	'c_A',	'd_A',	'g_A',
    'a_E',	'b_E',	'c_E',	'd_E',	'g_E',
    'a_I',	'b_I',	'c_I',	'd_I',	'g_I',
    'a_O',	'b_O',	'c_O',	'd_O',	'g_O',
    'a_U',	'b_U',	'c_U',	'd_U',	'g_U'
]
features_mfdfa = features[np.intersect1d(features.columns, names_mfdfa)]


In [11]:
names_surfboard = [
   'mfcc_mean_1_A', 'mfcc_mean_2_A', 'mfcc_mean_4_A', 'mfcc_mean_9_A',
       'mfcc_mean_10_A', 'mfcc_mean_12_A', 'mfcc_mean_13_A',
       'loudness_slidingwindow_mean_A', 'hnr_A', 'LPC_2_A', 'LPC_3_A',
       'mfcc_mean_1_E', 'mfcc_mean_3_E', 'mfcc_mean_5_E', 'mfcc_mean_6_E',
       'mfcc_mean_7_E', 'mfcc_mean_10_E', 'mfcc_mean_11_E', 'mfcc_mean_12_E',
       'mfcc_mean_13_E', 'f0_contour_mean_E', 'f0_mean_E', 'hnr_E', 'LPC_2_E',
       'LPC_3_E', 'mfcc_mean_1_I', 'mfcc_mean_3_I', 'mfcc_mean_5_I',
       'mfcc_mean_6_I', 'mfcc_mean_7_I', 'mfcc_mean_9_I', 'mfcc_mean_11_I',
       'mfcc_mean_12_I', 'mfcc_mean_13_I', 'f0_contour_mean_I', 'hnr_I',
       'LPC_2_I', 'LPC_3_I', 'LPC_4_I', 'mfcc_mean_3_O', 'mfcc_mean_4_O',
       'mfcc_mean_6_O', 'mfcc_mean_8_O', 'mfcc_mean_9_O', 'mfcc_mean_13_O',
       'f0_contour_mean_O', 'f0_std_O', 'log_energy_O',
       'loudness_slidingwindow_mean_O', 'mfcc_mean_2_U', 'mfcc_mean_3_U',
       'mfcc_mean_5_U', 'mfcc_mean_6_U', 'mfcc_mean_7_U', 'mfcc_mean_8_U',
       'mfcc_mean_10_U', 'mfcc_mean_11_U', 'hnr_U'
]
features_surfboard = features[np.intersect1d(features.columns, names_surfboard)]


In [12]:
# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)
features_mfdfa = np.array(features_mfdfa)
features_surfboard = np.array(features_surfboard)

# Random Forest

Se aplică Random Forest pentru 3 tipuri de date de intrare: separat doar separat MFDFA, doar Surfboard și ambele împreună. Se analizează acuratețea.

## MFDFA

In [31]:
X = features_mfdfa
y = labels

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
 np.unique(y_train, return_counts=True)
 np.unique(y_test, return_counts=True)

(array([0, 1]), array([29, 31]))

In [34]:
pipeline = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, max_leaf_nodes=10))
strtfdKFold = StratifiedKFold(n_splits=10)
kfold = strtfdKFold.split(X_train, y_train)
scores = []

for k, (train, test) in enumerate(kfold):
    pipeline.fit(X_train[train, :], y_train[train])
    score = pipeline.score(X_train[test, :], y_train[test])
    scores.append(score)
    print('Fold: %2d, Training/Test Split Distribution: %s, Accuracy: %.3f' % (k+1, np.bincount(y_train[train]), score))

print('\n\nCross-Validation accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

Fold:  1, Training/Test Split Distribution: [109 107], Accuracy: 0.708
Fold:  2, Training/Test Split Distribution: [109 107], Accuracy: 0.792
Fold:  3, Training/Test Split Distribution: [109 107], Accuracy: 0.958
Fold:  4, Training/Test Split Distribution: [109 107], Accuracy: 0.708
Fold:  5, Training/Test Split Distribution: [109 107], Accuracy: 0.833
Fold:  6, Training/Test Split Distribution: [109 107], Accuracy: 0.875
Fold:  7, Training/Test Split Distribution: [109 107], Accuracy: 0.667
Fold:  8, Training/Test Split Distribution: [109 107], Accuracy: 0.833
Fold:  9, Training/Test Split Distribution: [109 107], Accuracy: 0.750
Fold: 10, Training/Test Split Distribution: [108 108], Accuracy: 0.792


Cross-Validation accuracy: 0.792 +/- 0.083


## Surfboard

In [39]:
X = features_surfboard
y = labels

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [41]:
 np.unique(y_train, return_counts=True)
 np.unique(y_test, return_counts=True)

(array([0, 1]), array([29, 31]))

In [42]:
pipeline = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, max_leaf_nodes=10))
strtfdKFold = StratifiedKFold(n_splits=10)
kfold = strtfdKFold.split(X_train, y_train)
scores = []

for k, (train, test) in enumerate(kfold):
    pipeline.fit(X_train[train, :], y_train[train])
    score = pipeline.score(X_train[test, :], y_train[test])
    scores.append(score)
    print('Fold: %2d, Training/Test Split Distribution: %s, Accuracy: %.3f' % (k+1, np.bincount(y_train[train]), score))

print('\n\nCross-Validation accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

Fold:  1, Training/Test Split Distribution: [109 107], Accuracy: 0.917
Fold:  2, Training/Test Split Distribution: [109 107], Accuracy: 0.875
Fold:  3, Training/Test Split Distribution: [109 107], Accuracy: 0.875
Fold:  4, Training/Test Split Distribution: [109 107], Accuracy: 0.708
Fold:  5, Training/Test Split Distribution: [109 107], Accuracy: 0.917
Fold:  6, Training/Test Split Distribution: [109 107], Accuracy: 0.875
Fold:  7, Training/Test Split Distribution: [109 107], Accuracy: 0.875
Fold:  8, Training/Test Split Distribution: [109 107], Accuracy: 0.792
Fold:  9, Training/Test Split Distribution: [109 107], Accuracy: 0.875
Fold: 10, Training/Test Split Distribution: [108 108], Accuracy: 0.792


Cross-Validation accuracy: 0.850 +/- 0.062


## MFDFA + SURFBOARD

In [43]:
X = features
y = labels

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [45]:
 np.unique(y_train, return_counts=True)
 np.unique(y_test, return_counts=True)

(array([0, 1]), array([29, 31]))

In [46]:

pipeline = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, max_leaf_nodes=10))
strtfdKFold = StratifiedKFold(n_splits=10)
kfold = strtfdKFold.split(X_train, y_train)
scores = []

for k, (train, test) in enumerate(kfold):
    pipeline.fit(X_train[train, :], y_train[train])
    score = pipeline.score(X_train[test, :], y_train[test])
    scores.append(score)
    print('Fold: %2d, Training/Test Split Distribution: %s, Accuracy: %.3f' % (k+1, np.bincount(y_train[train]), score))

print('\n\nCross-Validation accuracy: %.3f +/- %.3f' %(np.mean(scores), np.std(scores)))

Fold:  1, Training/Test Split Distribution: [109 107], Accuracy: 0.917
Fold:  2, Training/Test Split Distribution: [109 107], Accuracy: 0.833
Fold:  3, Training/Test Split Distribution: [109 107], Accuracy: 0.875
Fold:  4, Training/Test Split Distribution: [109 107], Accuracy: 0.792
Fold:  5, Training/Test Split Distribution: [109 107], Accuracy: 0.792
Fold:  6, Training/Test Split Distribution: [109 107], Accuracy: 0.917
Fold:  7, Training/Test Split Distribution: [109 107], Accuracy: 0.750
Fold:  8, Training/Test Split Distribution: [109 107], Accuracy: 0.833
Fold:  9, Training/Test Split Distribution: [109 107], Accuracy: 0.833
Fold: 10, Training/Test Split Distribution: [108 108], Accuracy: 0.875


Cross-Validation accuracy: 0.842 +/- 0.052


# XGBoost

Se aplică XGBoost pentru 3 tipuri de date de intrare: separat doar separat MFDFA, doar Surfboard și ambele împreună. Se analizează acuratețea.

## MFDFA

In [25]:
X = features_mfdfa
y = labels

In [26]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert your data into DMatrix format (optimized data structure for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for XGBoost
params = {
    'max_depth': 3,  # maximum depth of a tree
    'objective': 'binary:logistic',  # objective function for binary classification
    'eval_metric': 'logloss',  # evaluation metric
    'eta': 0.1,  # learning rate
    'gamma': 0.1  # regularization parameter
}

# Train the model
num_rounds = 100  # number of boosting rounds
model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test set
preds = model.predict(dtest)
preds_binary = [1 if p >= 0.5 else 0 for p in preds]  # convert probabilities to binary labels (0 or 1)

# Evaluate the model
accuracy = accuracy_score(y_test, preds_binary)
auc = roc_auc_score(y_test, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("AUC: %.4f" % auc)

Accuracy: 76.67%
AUC: 0.8231


## Surfboard

In [27]:
X = features_surfboard
y = labels

In [28]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert your data into DMatrix format (optimized data structure for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for XGBoost
params = {
    'max_depth': 4,  # maximum depth of a tree
    'objective': 'binary:logistic',  # objective function for binary classification
    'eval_metric': 'logloss',  # evaluation metric
    'eta': 0.1,  # learning rate
    'gamma': 0.1  # regularization parameter
}

# Train the model
num_rounds = 100  # number of boosting rounds
model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test set
preds = model.predict(dtest)
preds_binary = [1 if p >= 0.5 else 0 for p in preds]  # convert probabilities to binary labels (0 or 1)

# Evaluate the model
accuracy = accuracy_score(y_test, preds_binary)
auc = roc_auc_score(y_test, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("AUC: %.4f" % auc)

Accuracy: 85.00%
AUC: 0.9488


## MFDFA + Surfboard

In [29]:
X = features
y = labels

In [30]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert your data into DMatrix format (optimized data structure for XGBoost)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set the parameters for XGBoost
params = {
    'max_depth': 4,  # maximum depth of a tree
    'objective': 'binary:logistic',  # objective function for binary classification
    'eval_metric': 'logloss',  # evaluation metric
    'eta': 0.1,  # learning rate
    'gamma': 0.1  # regularization parameter
}

# Train the model
num_rounds = 100  # number of boosting rounds
model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test set
preds = model.predict(dtest)
preds_binary = [1 if p >= 0.5 else 0 for p in preds]  # convert probabilities to binary labels (0 or 1)

# Evaluate the model
accuracy = accuracy_score(y_test, preds_binary)
auc = roc_auc_score(y_test, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("AUC: %.4f" % auc)

Accuracy: 86.67%
AUC: 0.9722
